In [35]:
import pandas as pd
import numpy as np
import itertools
from scipy import stats
from tqdm import tqdm

In [36]:
def move_column(df, cn, cp):
    # 定义移动的列
    m_column = df.pop(cn)
    # 关键代码(位置, 列名, 要移动的列) 
    df.insert(cp, cn, m_column)
    return df
    # 打印提示信息
    print('列位置更改成功')

# # 调用方法
# move_column(file_move, column_name, column_position)

In [37]:
# ━━━━━━━━━━━━━━━━━━━━━━━━  打开影像组学文件
df_radiomics = pd.read_table('20_RiskScore_radiomics1402_pretreated_OS.csv',sep=',',index_col=0)
feature_not_radiomics = df_radiomics.columns.tolist()[:3]
print(feature_not_radiomics)
df_radiomics

['status', 'time', 'RiskScore_h0']


,status,time,RiskScore_h0,log.sigma.5.mm.3D_glcm_Idn,squareroot_firstorder_Median,lbp.3D.k_glrlm_ShortRunEmphasis,wavelet.LLH_glszm_GrayLevelNonUniformity,log.sigma.5.mm.3D_firstorder_90Percentile,wavelet.LHH_ngtdm_Contrast,square_glrlm_RunEntropy,...,original_shape_MinorAxisLength,wavelet.LLH_firstorder_Mean,original_shape_Sphericity,wavelet.LLL_glcm_Idmn,wavelet.LHL_gldm_DependenceVariance,wavelet.HLL_gldm_DependenceVariance,log.sigma.3.mm.3D_glszm_ZoneEntropy,log.sigma.5.mm.3D_firstorder_Median,lbp.2D_firstorder_InterquartileRange,wavelet.LHH_gldm_DependenceNonUniformityNormalized
X0907236_ZhouJiSheng,1,6.83,0.896298,0.978929,418.103580,0.419132,73.934508,13.716698,0.001638,4.957925,...,56.688722,-1.006697,0.498970,0.998059,29.992383,33.763932,6.205362,-24.819478,2.5,0.104869
X0875917_LiYan,1,15.70,0.901296,0.976438,469.206017,0.372534,48.381674,18.177860,0.000817,4.973058,...,59.609625,-0.910070,0.662562,0.998251,33.161329,33.761676,6.552695,-29.572262,2.0,0.078167
TCGA-27-1830,1,5.13,2.621098,0.976809,361.326537,0.392058,91.859655,-1.832864,0.003141,4.811214,...,33.948305,-2.664174,0.616121,0.997026,28.714363,30.516095,5.870566,-48.545515,2.0,0.080178
TCGA-DU-5854,0,8.57,0.627098,0.975821,378.441206,0.297715,13.598095,9.866241,0.001674,4.827622,...,42.470084,-0.894696,0.663250,0.997674,27.613160,34.625264,5.097847,-20.290770,2.0,0.080079
HF1588,0,74.20,1.181956,0.968646,389.782128,0.409873,52.251438,12.997869,0.019593,4.761555,...,37.550941,-0.326498,0.531358,0.996102,25.230655,27.546266,5.379736,-40.516292,3.0,0.078900
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
X1052598_ShiHongMei,0,29.67,0.619839,0.973971,545.344729,0.405578,66.154018,-6.453781,0.001158,5.066364,...,51.721233,-1.941097,0.682478,0.998560,30.617479,34.898009,6.441233,-57.654155,3.0,0.085675
X0930877_XuPingLan,1,8.63,1.547634,0.980352,496.023289,0.296317,31.711602,19.271812,0.010592,5.255231,...,53.134315,-0.414315,0.613724,0.998826,33.895249,34.235136,6.592257,-24.207621,2.0,0.081979
TCGA-DU-6395,1,49.70,2.314271,0.977762,416.562123,0.339940,437.999479,4.845201,0.002410,4.611289,...,54.899323,-1.953735,0.373961,0.996877,33.189645,39.412890,4.749348,-26.371862,2.0,0.072793
HF1587,0,74.20,1.226855,0.972784,447.538339,0.349452,85.220925,6.252650,0.001351,4.968237,...,41.525773,-1.323163,0.691430,0.997400,30.526584,34.167278,5.717379,-53.489113,2.0,0.079584


In [38]:
# ━━━━━━━━━━━━━━━━━━━━━━━━  打开CIBERSORT结果文件
df_cibersort = pd.read_table(r'../1.CIBERSORT本地计算/TcgaRbrtJsph_cibersort.csv',sep=',',index_col=0)
move_column(df_cibersort, 'Macrophages M2', 0)
move_column(df_cibersort, 'Macrophages M1', 0)
move_column(df_cibersort, 'Macrophages M0', 0)

# 新增一列Macrophages 
MacrophageAll_df_orig = df_cibersort[['Macrophages M0','Macrophages M1','Macrophages M2']]
MacrophageAll_list = []
for row in MacrophageAll_df_orig.index:
    sum = 0
    for col in MacrophageAll_df_orig.columns:
        sum += MacrophageAll_df_orig.loc[row,col]
    MacrophageAll_list.append(sum)
    
df_cibersort.insert(3, 'Macrophages', MacrophageAll_list)  # 新列

# p_threshold = 0.05  # 筛选p值
# df_cibersort = df_cibersort[df_cibersort["P-value"] <= p_threshold]

feature_not_cibersort = df_cibersort.columns.tolist()[-3:]
print(feature_not_cibersort)
df_cibersort

['P-value', 'Correlation', 'RMSE']


,Macrophages M0,Macrophages M1,Macrophages M2,Macrophages,B cells naive,B cells memory,Plasma cells,T cells CD8,T cells CD4 naive,T cells CD4 memory resting,...,Monocytes,Dendritic cells resting,Dendritic cells activated,Mast cells resting,Mast cells activated,Eosinophils,Neutrophils,P-value,Correlation,RMSE
X1002249_GuCongXiang,0.00000,0.010115,0.472494,0.482609,0.000000,0.013321,0.028396,0.048041,0.0,0.034102,...,0.243043,0.000000,0.007005,0.000000,0.067758,0.006636,0.000000,0.04,0.149711,1.054281
X1010577_LiXiuJin,0.05367,0.007995,0.216151,0.277817,0.000000,0.044206,0.000000,0.078798,0.0,0.064831,...,0.081034,0.000000,0.029914,0.212367,0.000000,0.000000,0.000000,0.38,0.030343,1.069007
X1027770_LiuYueQin,0.05499,0.020090,0.342251,0.417332,0.002968,0.005006,0.006211,0.067801,0.0,0.060271,...,0.095318,0.000000,0.000000,0.188379,0.000000,0.000038,0.000000,0.33,0.038069,1.075370
X1029842_XuLongSheng,0.00000,0.002645,0.451138,0.453783,0.000000,0.004771,0.051071,0.119777,0.0,0.000000,...,0.081126,0.000000,0.029672,0.089350,0.000000,0.000000,0.007439,0.04,0.133902,1.038183
X1031574_LiLinSheng,0.00000,0.000000,0.376490,0.376490,0.063962,0.000000,0.042858,0.113963,0.0,0.000000,...,0.196162,0.000000,0.032493,0.000000,0.001895,0.000000,0.000000,0.07,0.102122,1.052912
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
NT3,0.00000,0.008581,0.159636,0.168218,0.000000,0.162707,0.000000,0.199642,0.0,0.003287,...,0.000000,0.103487,0.016694,0.000000,0.028079,0.000000,0.120564,0.13,0.108169,1.058147
NT4,0.00000,0.008006,0.202743,0.210749,0.070798,0.000000,0.000000,0.179094,0.0,0.000000,...,0.008488,0.089075,0.000000,0.000000,0.086813,0.000000,0.180117,0.02,0.243004,0.991865
NT5,0.00000,0.030730,0.056159,0.086889,0.028319,0.000000,0.000000,0.237129,0.0,0.000000,...,0.000000,0.133045,0.000000,0.000000,0.229040,0.000000,0.045461,0.05,0.148217,1.050252
NT6,0.00000,0.012482,0.177674,0.190156,0.055181,0.000000,0.000000,0.187551,0.0,0.000000,...,0.103842,0.092328,0.000000,0.000000,0.124618,0.001697,0.092690,0.02,0.241654,0.988483


In [39]:
# ━━━━━━━━━━━━━━━━━━━━━━━━  获取交叉
df_cibersort_list = df_cibersort.index.tolist()
df_radiomics_list = df_radiomics.index.tolist()
patCrossCondition = list(set(df_radiomics_list) & set(df_cibersort_list))
print(f'有{len(patCrossCondition)}个交叉')

df_tmp = pd.concat([df_cibersort,df_radiomics],axis=1).T[patCrossCondition]
df_tmp

有253个交叉


,TCGA-DU-7294,TCGA-06-0190,HF0960,HF1490,TCGA-76-4931,TCGA-CS-6186,TCGA-HT-7692,TCGA-HT-7884,HF1538,TCGA-HT-A617,...,TCGA-HT-7475,TCGA-CS-5397,HF1540,HF1381,TCGA-06-0645,TCGA-06-5413,TCGA-DU-6542,900-00-5381,HF1344,HF1150
Macrophages M0,0.088087,0.296274,0.000000,0.000000,0.467924,0.028983,0.000000,0.005397,0.000000,0.007719,...,0.105104,0.000000,0.342051,0.000000,0.012754,0.000000,0.000000,0.058007,0.000000,0.000000
Macrophages M1,0.045557,0.011206,0.001049,0.060574,0.018356,0.034458,0.003726,0.012531,0.029407,0.043700,...,0.004290,0.021075,0.015857,0.011708,0.013871,0.008899,0.010522,0.007325,0.033581,0.003276
Macrophages M2,0.117944,0.289897,0.151557,0.298863,0.236686,0.201018,0.397200,0.098948,0.220487,0.579588,...,0.005599,0.222944,0.171055,0.329346,0.353303,0.566357,0.349196,0.577233,0.357200,0.212343
Macrophages,0.251589,0.597377,0.152605,0.359437,0.722966,0.264459,0.400926,0.116876,0.249894,0.631007,...,0.114993,0.244018,0.528963,0.341054,0.379928,0.575256,0.359718,0.642565,0.390781,0.215619
B cells naive,0.005303,0.003225,0.053136,0.040431,0.000000,0.000000,0.017692,0.034621,0.034582,0.021428,...,0.000000,0.001415,0.031080,0.053345,0.006315,0.002124,0.000000,0.017483,0.032173,0.000000
B cells memory,0.001690,0.002085,0.000000,0.000000,0.037563,0.043474,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.009287,0.000000,0.026619,0.005748,0.025681,0.004292,0.000000,0.000000,0.007232
Plasma cells,0.000000,0.003056,0.000000,0.001766,0.007917,0.000000,0.000000,0.000000,0.000000,0.002604,...,0.000000,0.000000,0.000000,0.000000,0.001687,0.014491,0.000000,0.001561,0.000000,0.047737
T cells CD8,0.000000,0.000000,0.194417,0.119986,0.006808,0.024171,0.000000,0.056090,0.109943,0.056899,...,0.030196,0.037336,0.154398,0.267380,0.000000,0.030469,0.000000,0.096843,0.246523,0.096684
T cells CD4 naive,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.016054,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
T cells CD4 memory resting,0.257513,0.075743,0.000000,0.056105,0.020744,0.150064,0.106198,0.000000,0.059646,0.160004,...,0.299256,0.284639,0.000000,0.017704,0.096177,0.097699,0.143079,0.073900,0.007021,0.172214


In [40]:
x = list(df_tmp.index[:23])  # 免疫细胞富集
y = list(df_tmp.index[23+6:])  # 特征
retDic_cor = dict([(k,[]) for k in x])
retDic_p = dict([(k,[]) for k in x])

print(len(y))
y

20


['log.sigma.5.mm.3D_glcm_Idn',
 'squareroot_firstorder_Median',
 'lbp.3D.k_glrlm_ShortRunEmphasis',
 'wavelet.LLH_glszm_GrayLevelNonUniformity',
 'log.sigma.5.mm.3D_firstorder_90Percentile',
 'wavelet.LHH_ngtdm_Contrast',
 'square_glrlm_RunEntropy',
 'wavelet.HLL_firstorder_Kurtosis',
 'wavelet.HLL_glszm_SizeZoneNonUniformityNormalized',
 'wavelet.LHL_glszm_ZoneEntropy',
 'original_shape_MinorAxisLength',
 'wavelet.LLH_firstorder_Mean',
 'original_shape_Sphericity',
 'wavelet.LLL_glcm_Idmn',
 'wavelet.LHL_gldm_DependenceVariance',
 'wavelet.HLL_gldm_DependenceVariance',
 'log.sigma.3.mm.3D_glszm_ZoneEntropy',
 'log.sigma.5.mm.3D_firstorder_Median',
 'lbp.2D_firstorder_InterquartileRange',
 'wavelet.LHH_gldm_DependenceNonUniformityNormalized']

# 做相关：多维表

In [41]:
method = 'pearson' # method : {'pearson', 'spearman', 'kendall'} # 选择⽅法
list_retAndp = []

for i in tqdm(range(len(x))):
    fea = x[i]
    for rn in y:
        varSet1 = df_tmp.loc[fea].values
        varSet2 = df_tmp.loc[rn].values
        
        # 选择⽅法进⾏计算
        if method == 'pearson':
            r, p = stats.pearsonr(varSet1, varSet2)
        elif method == 'spearman':
            r, p = stats.spearmanr(varSet1, varSet2, nan_policy='omit')
        elif method == 'kendall':
            r, p = stats.kendalltau(varSet1, varSet2, nan_policy='omit')
        else:
            print("method⽅法选择错误")
            
        # 向相关度表中添加内容
        list_retAndp.append([fea, rn, r, p])

100%|█████████████████████████████████████████████████████████████████████████████████| 23/23 [00:00<00:00, 572.86it/s]


In [42]:
df_retAndp = pd.DataFrame(list_retAndp,columns=['ImmuneCell','RFs','R','pvalue'])
df_retAndp

,ImmuneCell,RFs,R,pvalue
0,Macrophages M0,log.sigma.5.mm.3D_glcm_Idn,0.042418,0.501797
1,Macrophages M0,squareroot_firstorder_Median,-0.085613,0.174618
2,Macrophages M0,lbp.3D.k_glrlm_ShortRunEmphasis,-0.062157,0.324756
3,Macrophages M0,wavelet.LLH_glszm_GrayLevelNonUniformity,-0.159030,0.011304
4,Macrophages M0,log.sigma.5.mm.3D_firstorder_90Percentile,0.027737,0.660595
...,...,...,...,...
455,Neutrophils,wavelet.HLL_gldm_DependenceVariance,-0.233523,0.000178
456,Neutrophils,log.sigma.3.mm.3D_glszm_ZoneEntropy,0.146399,0.019825
457,Neutrophils,log.sigma.5.mm.3D_firstorder_Median,0.077099,0.221678
458,Neutrophils,lbp.2D_firstorder_InterquartileRange,0.147580,0.018841


In [43]:
save_name3 = '→多维_相关系数表.csv'

df_retAndp.to_csv(save_name3, index=True)